In [2]:
from flask import Flask, request, jsonify,render_template
import numpy as np
import tensorflow as tf
import keras
from PIL import Image
import io

IMG_SIZE = 224  

app = Flask(__name__)

ALLOWED_EXTENSIONS = {"png", "jpg", "jpeg"}

model = keras.models.load_model("Plant_Village_Detection_Model.h5")

CLASS_LABELS = ['Apple___Apple_scab', 'Apple___Black_rot' ,'Apple___Cedar_apple_rust'
 'Apple___healthy', 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight'
 'Corn_(maize)___healthy' 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot',
 'Peach___healthy', 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy' 'Potato___Early_blight' 'Potato___Late_blight'
 'Potato___healthy' ,'Raspberry___healthy', 'Soybean___healthy',
 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch',
 'Strawberry___healthy' ,'Tomato___Bacterial_spot' 'Tomato___Early_blight',
 'Tomato___Late_blight', 'Tomato___Leaf_Mold' 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

def allowed_file(filename):
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')


@app.route("/predict", methods=["POST"])
def predict():
    try:
        if "image" not in request.files:
            return jsonify({"error": "يجب إرسال صورة"}), 400

        file = request.files["image"]

        if not allowed_file(file.filename):
            return jsonify({"error": "صيغة الملف غير مدعومة. استخدم PNG أو JPG فقط"}), 400

        try:
            image = Image.open(file.stream).convert("RGB")  
            image = image.resize((IMG_SIZE, IMG_SIZE)) 
            image = np.array(image) / 255.0  # **تحجيم القيم بين 0 و 1**
            image = np.expand_dims(image, axis=0)  # **إضافة بعد إضافي لتوافق الإدخال مع الموديل**

            # **تنفيذ التنبؤ باستخدام الموديل**
            prediction = model.predict(image)
            probabilities = prediction[0]  # مصفوفة الاحتمالات لكل فئة

            # **تحديد الفئة ذات أعلى احتمال**
            predicted_index = np.argmax(probabilities)  
            predicted_label = CLASS_LABELS[predicted_index]  
            confidence = probabilities[predicted_index] * 100  # تحويل الاحتمال إلى نسبة %

            return jsonify({
                "prediction": predicted_label,
                "probability": f"{confidence:.2f}%"  # عرض النسبة بدقة رقمين عشريين
            })

        except Exception as e:
            return jsonify({"error": f"فشل في معالجة الصورة: {str(e)}"}), 400

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8000, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.1.8:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.1.8 - - [17/Feb/2025 12:31:22] "POST /predict HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2025 12:31:38] "POST /predict HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2025 12:31:47] "POST / HTTP/1.1" 405 -
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-package